In [2]:
%load_ext autoreload
%autoreload 2

# install libraries that are not part of Colab by default
!pip install census us mapclassify -q

from maptools import ui, census_vars
from IPython.display import Markdown as md
from census import Census
import us
import pandas as pd
import geopandas as gpd
import requests



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
states = gpd.read_file(
    "https://www2.census.gov/geo/tiger/TIGER2022/STATE/tl_2022_us_state.zip")
states

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62266456923,489045863,+38.6472854,-080.6183274,"POLYGON ((-77.75438 39.33346, -77.75422 39.333..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138962819934,45971472526,+28.3989775,-082.5143005,"MULTIPOLYGON (((-83.10874 24.62949, -83.10711 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143778515726,6216539665,+40.1028754,-089.1526108,"POLYGON ((-87.89243 38.28285, -87.89334 38.282..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206244837557,18937184315,+46.3159573,-094.1996043,"POLYGON ((-95.31989 48.99892, -95.31747 48.998..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25151771744,6979295311,+38.9466584,-076.6744939,"POLYGON ((-75.756 39.24607, -75.75578 39.24334..."
5,1,1,44,01219835,44,RI,Rhode Island,00,G4000,A,2677763373,1323686975,+41.5964850,-071.5264901,"MULTIPOLYGON (((-71.67881 41.15891, -71.67626 ..."
6,4,8,16,01779783,16,ID,Idaho,00,G4000,A,214049908397,2391592787,+44.3484222,-114.5588538,"POLYGON ((-116.35844 49.00047, -116.34805 49.0..."
7,1,1,33,01779794,33,NH,New Hampshire,00,G4000,A,23190126218,1025960758,+43.6726907,-071.5843145,"POLYGON ((-70.83887 43.24449, -70.83886 43.244..."
8,3,5,37,01027616,37,NC,North Carolina,00,G4000,A,125935585728,13453835222,+35.5397100,-079.1308636,"POLYGON ((-77.89977 36.5446, -77.89942 36.5446..."
9,1,1,50,01779802,50,VT,Vermont,00,G4000,A,23872569964,1030754610,+44.0589536,-072.6710173,"POLYGON ((-72.04187 44.15665, -72.0418 44.1566..."


In [13]:
census_vars.search("educational attainment")

,group,concept,match
25034,B25013,Tenure By Educational Attainment Of Householder,73.83%
17497,B26206,Group Quarters Type (5 Types) By Educational Attainment,65.17%
11906,B26106,Group Quarters Type (3 Types) By Educational Attainment,65.17%
22279,B15003,Educational Attainment For The Population 25 Years And Over,64.39%
15569,B15002,Sex By Educational Attainment For The Population 25 Years And Over,61.16%
17638,B15001,Sex By Age By Educational Attainment For The Population 18 Years And Over,60.75%
23603,B99151,Allocation Of Educational Attainment For The Population 25 Years And Over,60.36%
24341,B06009,Place Of Birth By Educational Attainment In The United States,57.98%
7162,B29002,"Citizen, Voting-Age Population By Educational Attainment",56.00%
2206,B17003,Poverty Status In The Past 12 Months Of Individuals By Sex By Educational Attainment,55.30%


In [54]:

# education attainment
url = f"https://api.census.gov/data/2022/acs/acs5?get=group(B15003)&ucgid=pseudo(0100000US$0400000)"
meta_url = "https://api.census.gov/data/2022/acs/acs5"

df = census_vars.get(url, meta_url)
# df.explore(column="total", cmap="Purples", legend=False)
df.sort_values("total", ascending=False).head(10)

,state,state_name,total,no_schooling_completed,nursery_school,kindergarten,1st_grade,2nd_grade,3rd_grade,4th_grade,...,regular_high_school_diploma,ged_or_alternative_credential,some_college_less_than_1_year,some_college_1_or_more_years_no_degree,associate_s_degree,bachelor_s_degree,master_s_degree,professional_school_degree,doctorate_degree,geometry
13,CA,California,26842698,880931,8821,13922,31306,68837,144716,87564,...,4837641,647858,1592200,3816058,2136258,5935292,2515475,687281,488925,"MULTIPOLYGON (((-119.99987 41.18397, -119.9998..."
25,TX,Texas,18910948,462389,5656,6688,19733,41591,84895,54952,...,3792935,833194,1221713,2733655,1435330,3919003,1598654,341512,242618,"POLYGON ((-98.42353 34.08284, -98.42235 34.082..."
1,FL,Florida,15579847,266243,6015,3457,7636,14074,31109,24139,...,3671306,645493,1026244,1961436,1567030,3154240,1308569,372331,202326,"MULTIPOLYGON (((-83.10874 24.62949, -83.10711 ..."
35,NY,New York,14021808,356478,5300,4404,7569,13213,31343,29568,...,2933147,561245,682351,1426247,1238616,3033763,1731771,435559,238322,"MULTIPOLYGON (((-74.72623 44.99486, -74.72257 ..."
18,PA,Pennsylvania,9128083,100550,2237,1381,2755,3027,7242,6465,...,2710001,348743,500374,921061,805392,1844355,884422,199768,152773,"POLYGON ((-75.5931 39.83772, -75.59319 39.8377..."
2,IL,Illinois,8746416,139881,2131,2368,2966,7349,14037,11681,...,1899480,316426,582890,1149453,727981,1924339,938097,214115,130899,"POLYGON ((-87.89243 38.28285, -87.89334 38.282..."
24,OH,Ohio,8097704,85033,1573,1022,1417,2296,4722,3820,...,2315749,310108,560069,1040670,715379,1515763,696826,148249,98487,"POLYGON ((-80.51915 40.90033, -80.51914 40.899..."
8,NC,North Carolina,7172112,94221,1710,1527,2297,5199,11370,8644,...,1505077,299329,449128,1010992,720557,1533552,648712,139534,107092,"POLYGON ((-77.89977 36.5446, -77.89942 36.5446..."
21,GA,Georgia,7158620,116098,1612,1713,2604,5582,12918,8688,...,1611827,331469,418865,985474,594639,1464589,669304,165134,108680,"POLYGON ((-81.09538 31.52098, -81.0971 31.5197..."
38,MI,Michigan,6938439,83826,995,1424,1136,1882,4445,3636,...,1694166,276452,535510,1025808,673750,1311608,627217,132170,89356,"POLYGON ((-87.1127 47.818, -87.0631 47.79999, ..."


,state,state_name,total,no_schooling_completed,nursery_school,kindergarten,1st_grade,2nd_grade,3rd_grade,4th_grade,...,regular_high_school_diploma,ged_or_alternative_credential,some_college_less_than_1_year,some_college_1_or_more_years_no_degree,associate_s_degree,bachelor_s_degree,master_s_degree,professional_school_degree,doctorate_degree,geometry
7,NH,New Hampshire,997353,7152,304,275,142,401,487,399,...,229881,41093,60160,114506,100541,236189,116378,19939,16942,"POLYGON ((-70.83887 43.24449, -70.83886 43.244..."
18,PA,Pennsylvania,9128083,100550,2237,1381,2755,3027,7242,6465,...,2710001,348743,500374,921061,805392,1844355,884422,199768,152773,"POLYGON ((-75.5931 39.83772, -75.59319 39.8377..."
2,IL,Illinois,8746416,139881,2131,2368,2966,7349,14037,11681,...,1899480,316426,582890,1149453,727981,1924339,938097,214115,130899,"POLYGON ((-87.89243 38.28285, -87.89334 38.282..."
24,OH,Ohio,8097704,85033,1573,1022,1417,2296,4722,3820,...,2315749,310108,560069,1040670,715379,1515763,696826,148249,98487,"POLYGON ((-80.51915 40.90033, -80.51914 40.899..."
5,RI,Rhode Island,771284,14016,153,129,724,874,1481,2728,...,173104,34107,47993,90907,63413,165487,80811,19272,14255,"MULTIPOLYGON (((-71.67881 41.15891, -71.67626 ..."
42,MT,Montana,758200,3363,25,49,52,37,202,204,...,175154,35784,54030,121486,72238,170549,59333,15762,11983,"POLYGON ((-105.05764 48.99923, -105.0341 48.99..."
8,NC,North Carolina,7172112,94221,1710,1527,2297,5199,11370,8644,...,1505077,299329,449128,1010992,720557,1533552,648712,139534,107092,"POLYGON ((-77.89977 36.5446, -77.89942 36.5446..."
21,GA,Georgia,7158620,116098,1612,1713,2604,5582,12918,8688,...,1611827,331469,418865,985474,594639,1464589,669304,165134,108680,"POLYGON ((-81.09538 31.52098, -81.0971 31.5197..."
11,DE,Delaware,700364,6745,100,104,228,372,414,753,...,184812,23317,43967,87027,58540,139213,75033,13180,13931,"POLYGON ((-75.50949 39.68652, -75.50942 39.686..."
38,MI,Michigan,6938439,83826,995,1424,1136,1882,4445,3636,...,1694166,276452,535510,1025808,673750,1311608,627217,132170,89356,"POLYGON ((-87.1127 47.818, -87.0631 47.79999, ..."


In [49]:
metadata = requests.get(meta_url).json()
vars_url = metadata["dataset"][0]["c_variablesLink"]
vars = requests.get(vars_url).json()


In [50]:

vars = vars["variables"]
vars

{'for': {'label': "Census API FIPS 'for' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-for',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'in': {'label': "Census API FIPS 'in' clause",
  'concept': 'Census API Geography Specification',
  'predicateType': 'fips-in',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True},
 'ucgid': {'label': 'Uniform Census Geography Identifier clause',
  'concept': 'Census API Geography Specification',
  'predicateType': 'ucgid',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True,
  'hasGeoCollectionSupport': True},
 'B24022_060E': {'label': 'Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations',
  'concept': 'Sex by Occupation and Median Earnings in the Past 12 Months (in 2022 Inflation-Adjusted Dollars) for the Full-Time, Year-Round Civilian Employed Population 16 Years and Over',
  'predicateType': 'int',
  'group': 'B24022',
  'limit': 0,
  'attr

In [45]:
census_vars.get_variables()["type"].value_counts()

type
int       27643
float       200
string       10
Name: count, dtype: int64

In [185]:

# vars = meta["dataset"]["c_variables"]
# vars = requests.get(vars).json()
vars_url = meta["dataset"][0]["c_variablesLink"]
geo_url = meta["dataset"][0]["c_geographyLink"]

# for k, v in meta["dataset"][0].items():
#     print(k, v)

vars = requests.get(vars_url).json()
vars = vars["variables"]
geo = requests.get(geo_url).json()


In [189]:

field_names = dict()
for k, v in vars.items():
    if k in json[0]:
        if "predicateOnly" in v and v["predicateOnly"]:
            field_names[k] = k
        else:
            field_names[k] = v["label"]
# data.rename(columns=field_names)
data.columns

Index(['B01003_001E', 'B01003_001EA', 'B01003_001M', 'B01003_001MA', 'GEO_ID',
       'NAME', 'ucgid'],
      dtype='object')

In [165]:
index_cols = dict()

def indexes(k, v):
    parts = v.split("!!")
    t = tuple(parts[:-1])
    index_cols.setdefault(t, []).append(k)


[indexes(k, v) for k, v in field_names.items()]
longest = max(len(k) for k in index_cols.keys())
keys = list(index_cols.keys())
keys.sort(key=lambda x: (len(x), x))

def make_multi(k, v):
    df = data[v].copy().reset_index(drop=True)

    def f(x):
        n = field_names[x].split("!!")[-1]
        t = k + (n, )
        if len(k) < longest:
            t = t + ("",) * (longest - len(k))
        return t
    
    t = [f(c) for c in v]

    ix = pd.MultiIndex.from_tuples(t)
    df.columns = ix
    return df


indexed_data = [make_multi(k, index_cols[k]) for k in keys]

pd.concat(indexed_data, axis=1)

,ucgid,Geography
0,0100000US,0100000US


In [3]:
url = "http://api.census.gov/data/2023/acs/acs1/subject/variables.json"
resp = requests.get(url)
json = resp.json()

df = pd.DataFrame(json['variables']).T

df["var_name"] = df.label.apply(census_vars.col_name)
col_map = df["var_name"].to_dict()
cols = set(data.columns) & set(col_map.keys())
cols = sorted(list(cols))

df[df.index.isin(cols)].reset_index()["label"].iloc[0]


'Estimate!!Percent!!Total households!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2023 INFLATION-ADJUSTED DOLLARS)!!$75,000 or more:!!Without an Internet subscription'

In [4]:

col_map['[["GEO_ID"'] = 'GEOID'
col_map["NAME"] = 'state'
cols = set(data.columns) & set(col_map.keys())
cols = sorted(list(cols))
table = data[list(cols)].copy()
table = table.rename(columns=col_map)
table.columns


Index(['state', 'households', 'has_one_or_more_types_of_computing_devices',
       'desktop_or_laptop',
       'desktop_or_laptop_with_no_other_type_of_computing_device',
       'smartphone', 'smartphone_with_no_other_type_of_computing_device',
       'tablet_or_other_portable_wireless_computer',
       'tablet_or_other_portable_wireless_computer_with_no_other_type_of_computing_device',
       'other_computer',
       'other_computer_with_no_other_type_of_computing_device', 'no_computer',
       'with_an_internet_subscription',
       'dial_up_with_no_other_type_of_internet_subscription',
       'broadband_of_any_type', 'cellular_data_plan',
       'cellular_data_plan_with_no_other_type_of_internet_subscription',
       'broadband_such_as_cable_fiber_optic_or_dsl',
       'satellite_internet_service', 'without_an_internet_subscription',
       'less_than_20_000', 'with_dial_up_internet_subscription_alone',
       'with_a_broadband_internet_subscription',
       'without_an_internet_sub

In [5]:
url = "https://api.census.gov/data/2023/acs/acs1/subject?get=group(S2801)&ucgid=pseudo(0100000US$0400000)"
data = pd.read_csv(url)
table = census_vars.rename_columns(data)
table

,GEOID,NAME,households,has_one_or_more_types_of_computing_devices,desktop_or_laptop,desktop_or_laptop_with_no_other_type_of_computing_device,smartphone,smartphone_with_no_other_type_of_computing_device,tablet_or_other_portable_wireless_computer,tablet_or_other_portable_wireless_computer_with_no_other_type_of_computing_device,...,without_an_internet_subscription,20_000_to_74_999,with_dial_up_internet_subscription_alone,with_a_broadband_internet_subscription,without_an_internet_subscription,75_000_or_more,with_dial_up_internet_subscription_alone,with_a_broadband_internet_subscription,without_an_internet_subscription,ucgid]
0,"[""0400000US01""",Alabama,2051545,1940724,1479526,44395,1857116,297002,1215121,16485,...,24.9,-888888888,0.1,87.8,12.1,-888888888,0.1,96.4,3.5,0400000US01]
1,"[""0400000US02""",Alaska,276852,269648,229564,5292,260997,26347,178184,1174,...,20.1,-888888888,0.1,89.3,10.6,-888888888,0.1,97.6,2.3,0400000US02]
2,"[""0400000US04""",Arizona,2907014,2818465,2417106,64577,2708607,253699,1911929,14164,...,23.2,-888888888,0.2,90.5,9.2,-888888888,0.1,97.0,3.0,0400000US04]
3,"[""0400000US05""",Arkansas,1232871,1171045,878558,27121,1126328,208396,680511,7305,...,28.8,-888888888,0.1,87.8,12.1,-888888888,0.0,96.0,3.9,0400000US05]
4,"[""0400000US06""",California,13699816,13315679,11497837,244092,12885390,1125514,9255623,71173,...,18.8,-888888888,0.1,91.9,8.0,-888888888,0.1,97.6,2.4,0400000US06]
5,"[""0400000US08""",Colorado,2428261,2371266,2124065,46958,2292728,153919,1645746,9488,...,18.5,-888888888,0.2,92.0,7.8,-888888888,0.0,97.7,2.2,0400000US08]
6,"[""0400000US09""",Connecticut,1442969,1382529,1174225,39129,1312028,125086,944947,13373,...,24.6,-888888888,0.2,89.6,10.2,-888888888,0.2,97.3,2.5,0400000US09]
7,"[""0400000US10""",Delaware,412048,398260,329757,8175,381174,41133,269796,3545,...,22.3,-888888888,0.1,90.1,9.8,-888888888,0.0,97.3,2.7,0400000US10]
8,"[""0400000US11""",District of Columbia,334673,322682,288296,4824,313617,21718,217468,2405,...,28.9,-888888888,0.0,90.5,9.5,-888888888,0.0,97.3,2.7,0400000US11]
9,"[""0400000US12""",Florida,8966402,8707628,7360939,204867,8349941,802786,5935234,57846,...,18.8,-888888888,0.1,91.6,8.3,-888888888,0.0,96.8,3.2,0400000US12]


In [6]:
url = "https://api.census.gov/data/2022/acs/acs5?get=group(B03001)&ucgid=pseudo(0400000US36$8600000)"
zips = pd.read_csv(url)
table = census_vars.rename_columns(zips)
table

,B03001_002E,B03001_003E,B03001_004E,B03001_005E,B03001_006E,B03001_007E,B03001_008E,B03001_009E,B03001_010E,B03001_011E,...,B03001_025E,B03001_026E,B03001_027E,B03001_028E,B03001_029E,B03001_030E,B03001_031E,geography,NAME,ucgid]
0,1044,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12451,ZCTA5 12451,860Z200US12451]
1,3731,70,0,44,0,0,0,0,0,0,...,0,0,22,22,0,0,0,860Z200US13084,ZCTA5 13084,860Z200US13084]
2,313,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12856,ZCTA5 12856,860Z200US12856]
3,915,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US12985,ZCTA5 12985,860Z200US12985]
4,2869,9,0,2,0,0,0,0,0,0,...,0,0,6,0,6,0,0,860Z200US13332,ZCTA5 13332,860Z200US13332]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1350,19,2,17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,860Z200US14478,ZCTA5 14478,860Z200US14478]
1822,4324,454,0,349,0,0,42,0,21,0,...,0,0,63,28,0,0,35,860Z200US14510,ZCTA5 14510,860Z200US14510]
1823,4685,221,53,88,0,71,0,0,0,0,...,0,0,9,3,6,0,0,860Z200US14423,ZCTA5 14423,860Z200US14423]
1824,6917,1688,0,471,104,151,327,0,243,0,...,0,0,151,130,0,0,21,860Z200US10920,ZCTA5 10920,860Z200US10920]
